In [1]:
import pandas as pd
import re
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from konlpy.tag import Okt, Kkma, Mecab

pd.set_option('display.unicode.east_asian_width',True)

#중복 확인
df = pd.read_csv('./name_gender.csv')
col_dup = df['name'].duplicated()
print(col_dup)
sum_dup = df['name'].duplicated().sum()
print(sum_dup)

C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


0        False
1        False
2        False
3        False
4        False
         ...  
59899    False
59900    False
59901    False
59902    False
59903    False
Name: name, Length: 59904, dtype: bool
0


In [2]:
df = df.drop_duplicates(subset=['name'])
sum_dup = df['name'].duplicated().sum()
print(sum_dup)

df.reset_index(drop=True, inplace=True)

X = df['name']
Y = df['gender']


0


In [3]:
#라벨인코딩
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_
print(label)

#피클로 저장
with open('./model/gender_encoder.pickle', 'wb') as f:
    pickle.dump(encoder, f)

print(labeled_Y)

onehot_Y = to_categorical(labeled_Y)
print(onehot_Y)

################ okt 70초
# okt = Okt()
# print(type(X))
# okt_X = okt.morphs(X[0])
# print(X[0])
# print(okt_X)


# for i in range(len(X)):
#     X[i] = okt.morphs(X[i])
# print(X)


############### 꼬마 200초 이상
# kkma = Kkma()
# print(type(X))
# kkma_X = kkma.morphs(X[0])
# print(X[0])
# print(kkma_X)

# for i in range(len(X)):
#   X[i] = kkma.morphs(X[i])
# print(X)

############### 메캅 40초
mecab=Mecab(dicpath='C:/mecab/mecab-ko-dic')
print(type(X))
mecab_X = mecab.morphs(X[0])
print(X[0])
print(mecab_X)

for i in range(len(X)):
  X[i] = mecab.morphs(X[i])
print(X)


[1 2 3]
[0 0 0 ... 2 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
<class 'pandas.core.series.Series'>
민준
['민준']


C:\Users\kim\.conda\envs\exam_cv2\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0            [민준]
1            [서준]
2            [도윤]
3            [시우]
4            [주원]
            ...    
59899      [희, 해]
59900      [희, 활]
59901    [히, 까리]
59902    [히이, 로]
59903    [힐드, 로]
Name: name, Length: 59904, dtype: object


In [4]:
token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

with open('./model/gender_token_mecab.pickle', 'wb') as f:
    pickle.dump(token, f)

wordsize = len(token.word_index) + 1
print(wordsize)

max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[441]
25827
8
[[  0   0   0   0   0   0   0 441]
 [  0   0   0   0   0   0   0 532]
 [  0   0   0   0   0   0   0 996]
 [  0   0   0   0   0   0   0 354]
 [  0   0   0   0   0   0   0 427]
 [  0   0   0   0   0   0   0 428]
 [  0   0   0   0   0   0   0 292]
 [  0   0   0   0   0   0   0 416]
 [  0   0   0   0   0   0   0 506]
 [  0   0   0   0   0   0   0 482]]


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

xy = X_train, X_test, Y_train, Y_test
np.save('./model/gender_data_max_{}_size_{}'.format(max, wordsize), xy)

(47923, 8)
(11981, 8)
(47923, 3)
(11981, 3)
